In [1]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.
GPU run command with Theano backend (with TensorFlow, the GPU is automatically used):
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatx=float32 python cifar10_cnn.py
It gets down to 0.65 test logloss in 25 epochs, and down to 0.55 after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

batch_size = 32
num_classes = 10
epochs = 200
data_augmentation = True

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))
    
# saving model to file
model.save('cifar10.h5')

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/200
1562/1562 [==============================] - 194s - loss: 1.8559 - acc: 0.3199 - val_loss: 1.5613 - val_acc: 0.4371
Epoch 2/200
1562/1562 [==============================] - 194s - loss: 1.5793 - acc: 0.4251 - val_loss: 1.3926 - val_acc: 0.4985
Epoch 3/200
1562/1562 [==============================] - 194s - loss: 1.4527 - acc: 0.4772 - val_loss: 1.2762 - val_acc: 0.5415
Epoch 4/200
1562/1562 [==============================] - 194s - loss: 1.3630 - acc: 0.5113 - val_loss: 1.1945 - val_acc: 0.5694
Epoch 5/200
1562/1562 [==============================] - 194s - loss: 1.2885 - acc: 0.5415 - val_loss: 1.1340 - val_acc: 0.5979
Epoch 6/200
1562/1562 [==============================] - 194s - loss: 1.2311 - acc: 0.5605 - val_loss: 1.0464 - val_acc: 0.6302
Epoch 7/200
1562/1562 [==============================] - 194s - loss: 1.1772 - acc: 0.5831 - val_loss: 1.0174 - val_acc: 0.6

1562/1562 [==============================] - 192s - loss: 0.8799 - acc: 0.7154 - val_loss: 0.7199 - val_acc: 0.7711
Epoch 127/200
1562/1562 [==============================] - 192s - loss: 0.8851 - acc: 0.7142 - val_loss: 0.7375 - val_acc: 0.7737
Epoch 128/200
1562/1562 [==============================] - 192s - loss: 0.8929 - acc: 0.7148 - val_loss: 0.7416 - val_acc: 0.7605
Epoch 129/200
1562/1562 [==============================] - 192s - loss: 0.8865 - acc: 0.7127 - val_loss: 0.9003 - val_acc: 0.7313
Epoch 130/200
1562/1562 [==============================] - 192s - loss: 0.9057 - acc: 0.7083 - val_loss: 0.7351 - val_acc: 0.7659
Epoch 131/200
1562/1562 [==============================] - 192s - loss: 0.9001 - acc: 0.7098 - val_loss: 0.7942 - val_acc: 0.7516
Epoch 132/200
1562/1562 [==============================] - 192s - loss: 0.9117 - acc: 0.7066 - val_loss: 0.7956 - val_acc: 0.7564
Epoch 133/200
1562/1562 [==============================] - 192s - loss: 0.9152 - acc: 0.7055 - val_loss:

In [3]:
# loading model
from keras.models import load_model
del model

model = load_model('cifar10.h5')

In [59]:
def decode_prediction(results):
    
    if results[0][0]==1:
        result = "airplane"
    elif results[0][1]==1:
        result = "automobile"
    elif results[0][2]==1:
        result = "bird"
    elif results[0][3]==1:
        result = "cat"
    elif results[0][4]==1:
        result = "deer"
    elif results[0][5]==1:
        result = "dog"
    elif results[0][6]==1:
        result = "frog"
    elif results[0][7]==1:
        result = "horse"
    elif results[0][8]==1:
        result = "ship"
    elif results[0][9]==1:
        result = "truck"
    
    return result;

In [62]:
from keras.preprocessing import image
import numpy as np

img_path = 'thor.jpeg'
img = image.load_img(img_path, target_size=(32, 32))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
preds = model.predict(x)

#print(preds)
print(decode_prediction(preds))

dog
